In this demo file, we investigate methods for explainability in medical imaging using pre-trained TorchXRayVision models ( https://github.com/mlmed/torchxrayvision ).

# Setup

In [1]:
!pip install torchxrayvision captum gradio
!git clone https://github.com/mlmed/gifsplanation

from pathlib import Path
from typing import Callable

import gradio as gr
import numpy as np
import torchxrayvision as xrv
import skimage, torch, torchvision
import matplotlib.pyplot as plt
import sys,os
sys.path.insert(0,"./gifsplanation/")

from captum.attr import IntegratedGradients, Saliency, InputXGradient
import attribution

!wget https://raw.githubusercontent.com/mlmed/torchxrayvision/master/tests/16747_3_1.jpg #download test image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 29.0 MB 56.0 MB/s 
     |████████████████████████████████| 1.4 MB 27.6 MB/s 
     |████████████████████████████████| 5.3 MB 39.2 MB/s 
     |████████████████████████████████| 2.3 MB 35.1 MB/s 
     |████████████████████████████████| 57 kB 2.1 MB/s 
     |████████████████████████████████| 54 kB 1.5 MB/s 
     |████████████████████████████████| 212 kB 40.6 MB/s 
     |████████████████████████████████| 270 kB 44.9 MB/s 
     |████████████████████████████████| 112 kB 45.7 MB/s 
     |████████████████████████████████| 55 kB 1.2 MB/s 
     |████████████████████████████████| 84 kB 2.0 MB/s 
     |████████████████████████████████| 84 kB 2.5 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 80 kB 9.1 MB/s 
     |████████████████████████████████| 68 kB 3.4 MB/s 
     |████████████████████████████████| 46 kB 

In [2]:
def make_fig(plot_matrix):
    fig = plt.figure()
    plt.imshow(plot_matrix, cmap=plt.cm.hot)
    #plt.title(plot_title)
    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    return fig

In [3]:
def xrv_prepare_image(image):
    img = xrv.datasets.normalize(image, 255) # convert 8-bit image to [-1024, 1024] range
    img = img.mean(2)[None, ...] # Make single color channel
    transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])
    img = transform(img)
    img = torch.from_numpy(img)
    
    return img[None,...]

In [7]:
def predict(image, model_choice):
    """Function that serves predictions."""
    img = xrv_prepare_image(image)
    model = xrv.models.DenseNet(weights=model_choice)
    model.eval()

    outputs = model(img)
    scores =  outputs[0].detach().numpy().astype(np.float) #conversion to np.float is needed for visualization with gr.Label
    label = dict(zip(model.pathologies, scores) )
    return label

In [15]:
def explain_gradient(image, model_choice, target):
    """Function that serves explanations."""
    
    model = xrv.models.DenseNet(weights=model_choice)
    input = xrv_prepare_image(image)
    
    #Saliency
    saliency = Saliency(model)
    attr = saliency.attribute(input, target=model.pathologies.index(target))
    fig1 = make_fig( np.abs(attr[0,0].numpy()) )

    return fig1

def explain_input_x_gradient(image, model_choice, target):
    """Function that serves explanations."""
    
    model = xrv.models.DenseNet(weights=model_choice)
    input = xrv_prepare_image(image)
    
    #InputXGradient
    ixg = InputXGradient(model)
    attr = ixg.attribute(input, target=model.pathologies.index(target))
    fig2 = make_fig( np.abs(attr[0,0].detach().numpy()) ) 
    return fig2
    
def explain_integrated_gradients(image, model_choice, target):
    """Function that serves explanations."""
    
    model = xrv.models.DenseNet(weights=model_choice)
    input = xrv_prepare_image(image)
    
    #IntegratedGradients
    ig = IntegratedGradients(model)
    attr = ig.attribute(input, target=model.pathologies.index(target))
    fig3 = make_fig( np.abs(attr[0,0].detach().numpy()) )
    return fig3

def explain_gifsplanation(image, model_choice, target):
    """Function that serves explanations."""
    
    model = xrv.models.DenseNet(weights=model_choice)
    input = xrv_prepare_image(image)
    
    #Gifsplanation
    input.requires_grad=False
    ae = xrv.autoencoders.ResNetAE(weights="101-elastic")
    movie = attribution.generate_video(input, model, target, ae, target_filename="test", border=False, show=False,
                        ffmpeg_path="ffmpeg")

    return movie
  






In [9]:
model_choices = ["densenet121-res224-all", "densenet121-res224-rsna","densenet121-res224-nih","densenet121-res224-pc","densenet121-res224-chex","densenet121-res224-mimic_nb","densenet121-res224-mimic_ch"]

target_choices = ['Atelectasis','Consolidation','Infiltration','Pneumothorax','Edema','Emphysema',
                'Fibrosis','Effusion','Pneumonia','Pleural_Thickening','Cardiomegaly','Nodule','Mass',
                'Hernia','Lung Lesion','Fracture','Lung Opacity','Enlarged Cardiomediastinum']

In [17]:
# build a basic browser interface to a Python function
frontend = gr.Blocks()

with frontend:
  gr.Markdown(
    """
    # X-Ray Diagnosis
    Explore TorchXRayVision model predictions and data using this demo.
    """
    )
  #layout
  with gr.Tab("Prediction"):
    with gr.Row():
      with gr.Column():
        input_image = gr.Image(label="X-ray image")
        select_model = gr.Dropdown(label="Select model", choices=model_choices)
        with gr.Row():
          submit_button = gr.Button("Submit")
        gr.Examples(["16747_3_1.jpg"], inputs=input_image)

      with gr.Column():
        label = gr.Label(label="Multiclass predictions")

  submit_button.click(predict, [input_image, select_model], label) 

  with gr.Tab("Saliency Maps"):

    with gr.Row():

      with gr.Column():
        select_target = gr.Dropdown(label="Select target", choices=target_choices)
        with gr.Row():
          original_image = gr.Image(label='Original', interactive=False)
          gradient_plot = gr.Plot(label="Gradient")

        with gr.Row():
          input_x_gradient_plot = gr.Plot(label='InputXGradients')
          integrated_gradients_plot = gr.Plot(label="IntegratedGradients")

  input_image.change(lambda s: s, inputs=input_image, outputs=original_image)
  select_target.change(explain_gradient, inputs=[input_image,select_model, select_target], outputs=gradient_plot)
  select_target.change(explain_input_x_gradient, inputs=[input_image,select_model, select_target], outputs=input_x_gradient_plot)
  select_target.change(explain_integrated_gradients, inputs=[input_image,select_model, select_target], outputs=integrated_gradients_plot)

  with gr.Tab("Gifsplanation"):
    with gr.Row():
      with gr.Column():
        select_target2 = gr.Dropdown(label="Select target", choices=target_choices)
        with gr.Row():
          original_image = gr.Image(label='Original', interactive=False)
          gifsplanation_vid = gr.Video(label="Gifsplanation")

  input_image.change(lambda s: s, inputs=input_image, outputs=original_image)
  select_target2.change(explain_gifsplanation, inputs=[input_image,select_model, select_target2], outputs=gifsplanation_vid)

frontend.launch(share=True, show_error=True)

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://17058.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7f62248acc90>,
 'http://127.0.0.1:7861/',
 'https://17058.gradio.app')

In [16]:
frontend.close()

Closing server running on port: 7861
